## Import our dataset
- our dataset consists of several files:
    - train.csv: our training data
    - test.csv: our testing data
    - sample_submissions.csv: A sample submission file in the correct format (columns key and fare_amount). This dummy file 'predicts' fare_amount to be $11.35 for all rows, which is the mean fare_amount from the training set.

In [15]:
import pandas as pd
import os

DATA_FILES_PATH = 'projectDataFiles/'

def import_training_dataset(file_path, row_limit):
    """
    function to import the dataset into a pandas dataframe.

    Takes a row limit to limit the number of rows read.
    """
    return pd.read_csv(file_path, nrows=row_limit)

# assign the dataset to the TRAIN Dataframe, right now we are only loading 1,000,000 rows (possibly chunk and feather to reduce loading time)
TRAIN = import_training_dataset(f'{DATA_FILES_PATH}train.csv', 1000000)

# show the head of the the dataset to see its columns
TRAIN.head()
    

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


## Define a function to get the manhattan distance between two lat/long points
- manhattan distance should be fairly relistic to new york because of the ways that streets work there, but we might want to use the real travel distance between locations somehow

# Jose
using Google maps API might be over used due to size of data and will be charged

In [16]:
# PIP INSTALL GMAPS TO INSTALL GOOGLE MAPS
# run on command line and/or terminal: pip install gmaps
# NEW: pip install -U googlemaps

import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key='key')

# print(googlemaps)

#distance = gmaps.distance_matrix("Los Angeles", "Las Vegas")
# distance = gmaps.distance_matrix((34.0522, -118.2437), (36.1699, -115.1398),mode='driving')["rows"][0]["elements"][0]["distance"]["value"]
# print(distance)

# Geocoding an address
#geocode_result = gmaps.geocode('1600 Amphitheatre Parkway, Mountain View, CA')

# Look up an address with reverse geocoding
#reverse_geocode_result = gmaps.reverse_geocode((36.1699, -115.1398))
#print(reverse_geocode_result)

# Request directions via public transit
#now = datetime.now()
#directions_result = gmaps.directions("Sydney Town Hall",
                                    # "Parramatta, NSW",
                                    # mode="transit",
                                    # departure_time=now)

#print(directions_result)


0.011741999999998143

# Brandon
using distance formula

In [20]:
import math

def geo_manhattan_distance(x1,y1,x2,y2):
    """
    pickup_longitude   pickup_latitude   dropoff_longitude   dropoff_latitude
    x1                 y1                x2                  y2
    returns the manhattan distance between two geo points
    """
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
print("distance:")
print(geo_manhattan_distance(
    TRAIN['pickup_longitude'].iloc[0], TRAIN['pickup_latitude'].iloc[0],
    TRAIN['dropoff_longitude'].iloc[0], TRAIN['dropoff_latitude'].iloc[0]))

# test it out
# it works tested it with an online calulator 
# pickup_longitude   pickup_latitude   dropoff_longitude     dropoff_latitude
# -73.844311         40.721319         -73.841610            40.712278
# x1                 y1                x2                    y2
# distance: 0.00943584029114223



distance:
0.00943584029114223


insert distances to data frame

In [ ]:
# Distances list
Dis_list = []
pointAB = ['pickup_longitude','pickup_latitude',
            'dropoff_longitude','dropoff_latitude']

# get distances of each column
for i in range(len(TRAIN)):
    Dis_list.append(geo_manhattan_distance(
    TRAIN['pickup_longitude'].iloc[i], TRAIN['pickup_latitude'].iloc[i],
    TRAIN['dropoff_longitude'].iloc[i], TRAIN['dropoff_latitude'].iloc[i]))

# insert distance into panda
TRAIN.insert(8,"Distance",Dis_list,True)

TRAIN.head()